Установка библиотеки PySpark

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=49851191cbc2e0dd497a891c40cc83ec394a38c7de054c8313a9fa0ba7b92532
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Импортирование из библиотеки PySpark.SQL Spark сессии и функций

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, to_date, sum

Создание Spark сессии

In [30]:
spark = SparkSession.builder.appName('web_server_logs').getOrCreate()

Чтение данных из CSV-файла и вывод схемы DataFrame с таблицей

In [31]:
web_server_logs_df = spark.read.csv(path='web_server_logs.csv', header=True, inferSchema=True)
web_server_logs_df.printSchema()
web_server_logs_df.show(6)

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+-------------+--------------------+------+------------------+-------------+-------------+
|           ip|           timestamp|method|               url|response_code|response_size|
+-------------+--------------------+------+------------------+-------------+-------------+
|170.249.180.5|2024-02-22 14:01:...|DELETE|        wp-content|          404|         2120|
|16.178.228.38|2024-02-17 10:28:...|   GET|posts/search/posts|          301|         8908|
|29.100.104.74|2024-01-11 06:23:...|   PUT|   tags/categories|          500|         6252|
| 136.28.17.16|2024-07-05 06:13:...|   GET|   tag/tag/explore|          301|         7625|
|210.198.57.37|2024-05-14 02:09:...|   PUT|               tag|          200|         3289|
| 36.62.168.54|2024-04-

**Анализ информации**
1. Сгруппировать данные по IP, посчитать количество запросов для каждого IP и вывести 10 самых активных IP

In [32]:
print('Top 10 active IP addresses:')
web_server_logs_df.groupBy('ip').agg(count(col='ip').alias('request_count')).orderBy(col(col='request_count').desc()).limit(10).show()

Top 10 active IP addresses:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|   33.26.88.151|            2|
| 173.226.142.87|            1|
|171.216.138.171|            1|
|  202.31.216.12|            1|
|120.203.174.162|            1|
| 128.135.26.232|            1|
|  98.17.231.161|            1|
|  138.200.77.47|            1|
|    37.79.75.89|            1|
| 161.251.173.53|            1|
+---------------+-------------+



2. Сгруппировать данные по HTTP-методу и посчитать количество запросов для каждого метода

In [33]:
print('Request count by HTTP method:')
web_server_logs_df.groupBy('method').agg(count(col='method').alias('method_count')).show()

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       24903|
|DELETE|       25244|
|   PUT|       24983|
|   GET|       24870|
+------+------------+



3. Профильтровать и посчитать количество запросов с кодом ответа 404

In [34]:
print(f"Number of 404 response codes: {web_server_logs_df.where(condition=col(col='response_code') == 404).count()}")

Number of 404 response codes: 24845


4. Сгруппировать данные по дате, просуммирвать размер ответов и отсортировать по дате

In [35]:
print('Total response size by day:')
web_server_logs_df.withColumn(colName='date', col=to_date(col='timestamp', format='yyyy-MM-dd')).groupBy('date').agg(sum(col='response_size').alias('total_response_size')).orderBy('date').show()

Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            2237910|
|2024-01-02|            2325011|
|2024-01-03|            2333383|
|2024-01-04|            2348042|
|2024-01-05|            2468375|
|2024-01-06|            2397493|
|2024-01-07|            2504628|
|2024-01-08|            2346610|
|2024-01-09|            2450523|
|2024-01-10|            2433686|
|2024-01-11|            2559643|
|2024-01-12|            2299851|
|2024-01-13|            2456457|
|2024-01-14|            2395780|
|2024-01-15|            2403393|
|2024-01-16|            2285858|
|2024-01-17|            2192839|
|2024-01-18|            2511498|
|2024-01-19|            2363614|
|2024-01-20|            2265719|
+----------+-------------------+
only showing top 20 rows



Закрыть Spark сессию

In [36]:
spark.stop()